In [1]:
!pip install gradio langchain-google-genai langchain tensorflow pillow

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.7/788.7 kB 21.9 MB/s eta 0:00:00


In [1]:
import gradio as gr
import numpy as np
from PIL import Image
from dotenv import load_dotenv
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

/opt/anaconda3/envs/dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-06 21:39:37.777506: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# 3. Load environment variables
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GEMINI_MODEL = "gemini-2.0-flash"

In [6]:
# 4. Initialize Gemini model
llm = ChatGoogleGenerativeAI(
    google_api_key=GEMINI_API_KEY,
    model=GEMINI_MODEL,
    temperature=0.4
)

In [8]:
# 5. Load trained CNN model
cnn_model = load_model("saved_models/dementia_cnn_sequential_4_model_V2.keras")

In [ ]:
# # 6. Class label map
# label_map = {
#     0: "Non-dementia",
#     1: "Very mild dementia",
#     2: "Mild dementia",
#     3: "Moderate dementia"
# }

In [ ]:
# 7. Function: Classify MRI image
def predict_from_mri(image):
    image = image.resize((224, 224)).convert("RGB")
    array = img_to_array(image) / 255.0
    array = np.expand_dims(array, axis=0)
    preds = cnn_model.predict(array)
    label = label_map[np.argmax(preds)]
    return label


NameError: name 'array' is not defined

In [12]:
# 8. Function: Use Gemini to generate natural language explanation
def generate_gemini_summary(input_text):
    prompt = PromptTemplate.from_template("""
You are a dementia diagnostic assistant.

Interpret the following patient data or classification and explain what it means in simple, compassionate terms for a family or clinical audience:

"{input_text}"

Do not diagnose or suggest treatment. Just explain what it may indicate about dementia stages.
""")
    final_prompt = prompt.format(input_text=input_text)
    response = llm.invoke(final_prompt)
    return response.content.strip()

In [13]:
# 9. Combined chatbot function
def multimodal_chatbot(mri_image, symptom_text):
    responses = []

    if mri_image:
        diagnosis = predict_from_mri(mri_image)
        responses.append(f"**MRI Classification:** {diagnosis}")
        explanation = generate_gemini_summary(f"MRI indicates: {diagnosis}")
        responses.append(f"**Gemini Summary for MRI:**\n{explanation}")

    if symptom_text:
        explanation = generate_gemini_summary(f"Symptoms: {symptom_text}")
        responses.append(f"**Gemini Summary for Symptoms:**\n{explanation}")

    if not responses:
        return "Please upload an MRI image or enter symptom information."

    return "\n\n".join(responses)

In [14]:
# 10. Gradio UI
gr.Interface(
    fn=multimodal_chatbot,
    inputs=[
        gr.Image(type="pil", label="Upload MRI Image"),
        gr.Textbox(lines=4, placeholder="Describe symptoms here...", label="Symptom Description")
    ],
    outputs="markdown",
    title="Dementia MRI & Symptom Assistant (Powered by Gemini)",
    description="Upload an MRI or enter symptoms. The assistant will classify dementia and explain the findings using Google's Gemini 2.0 model."
).launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 648ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step
